## **Data Collection** 

In [ ]:
! git clone https://github.com/dasparagjyoti/Face_Mask_New.git

Cloning into 'Face_Mask_New'...
remote: Enumerating objects: 11093, done.
remote: Total 11093 (delta 0), reused 0 (delta 0), pack-reused 11093
Receiving objects: 100% (11093/11093), 251.83 MiB | 25.69 MiB/s, done.
Checking out files: 100% (11391/11391), done.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 64

DIRECTORY = r"/content/Face_Mask_New/Face_Mask_Detection/dataset"
CATEGORIES = [ "improper_mask", "with_mask", "without_mask"]

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

# data = []
# labels = []

# for category in CATEGORIES:
#     print("CATEGORY:", category)
#     path = os.path.join(DIRECTORY, category)
#     for img in os.listdir(path):
#       print("IMG", img)
#       img_path = os.path.join(path, img)
#       image = load_img(img_path, target_size=(224, 224))
#       image = img_to_array(image)
#       image = preprocess_input(image)

#       data.append(image)
#       labels.append(category)

# # perform one-hot encoding on the labels
# lb = LabelBinarizer()
# labels = lb.fit_transform(labels)

# data = np.array(data, dtype="float32")
# labels = np.array(labels)

# #split into train,test and validation 
# trainX, testX, trainY, testY = train_test_split(data,labels,test_size=0.2,train_size=0.8,stratify=labels, random_state=42)
# X_train, X_cv, Y_train, Y_cv = train_test_split(trainX,trainY,test_size = 0.25,train_size =0.75,stratify=trainY, random_state=42)


# construct the training image generator for data augmentation
train_datagen = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest",
  validation_split = 0.2
  )

val_datagen = ImageDataGenerator(
    validation_split = 0.2
)

train_data = train_datagen.flow_from_directory(
    directory = "/content/Face_Mask_New/Face_Mask_Detection/dataset",
    target_size = (224, 224),
    class_mode="categorical",
    batch_size=BS,
    shuffle=True,
    seed=42,
    subset = "training"
)
val_data = val_datagen.flow_from_directory(
    directory = "/content/Face_Mask_New/Face_Mask_Detection/dataset",
    target_size = (224, 224),
    class_mode="categorical",
    batch_size=BS,
    shuffle=True,
    seed=42,
    subset = "validation"
)

# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
for layer in baseModel.layers:
	layer.trainable = False


# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process


# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
	train_data,
	steps_per_epoch=train_data.__len__(),
	validation_data=val_data,
  epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
# predIdxs = model.predict(testX, batch_size=BS)
#predIdxs1 = model.predict(valX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
# predIdxs = np.argmax(predIdxs, axis=1)
# predIdxs1 = np.argmax(predIdxs1, axis=1)
# show a nicely formatted classification report
# print(classification_report(testY.argmax(axis=1), predIdxs,
	# target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("mask_detector.model", save_format="h5")



[INFO] loading images...
Found 9108 images belonging to 3 classes.
Found 2276 images belonging to 3 classes.
9412608/9406464 [==============================] - 0s 0us/step
[INFO] compiling model...
[INFO] training head...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
 99/194 [==============>...............] - ETA: 1:04 - loss: 1.0834 - accuracy: 0.4829

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


194/194 [==============================] - 173s 733ms/step - loss: 0.9294 - accuracy: 0.5753 - val_loss: 0.6296 - val_accuracy: 0.7544
Epoch 2/20
194/194 [==============================] - 141s 727ms/step - loss: 0.6725 - accuracy: 0.7161 - val_loss: 0.5275 - val_accuracy: 0.7909
Epoch 3/20
194/194 [==============================] - 141s 728ms/step - loss: 0.6021 - accuracy: 0.7538 - val_loss: 0.4966 - val_accuracy: 0.7935
Epoch 4/20
194/194 [==============================] - 141s 724ms/step - loss: 0.5620 - accuracy: 0.7690 - val_loss: 0.4729 - val_accuracy: 0.8049
Epoch 5/20
194/194 [==============================] - 139s 718ms/step - loss: 0.5261 - accuracy: 0.7901 - val_loss: 0.4881 - val_accuracy: 0.7931
Epoch 6/20
194/194 [==============================] - 140s 720ms/step - loss: 0.5058 - accuracy: 0.7984 - val_loss: 0.4127 - val_accuracy: 0.8423
Epoch 7/20
194/194 [==============================] - 140s 718ms/step - loss: 0.4926 - accuracy: 0.8052 - val_loss: 0.4435 - val_accura

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
